In [ ]:
SANDBOX_NAME = ''# Sandbox Name
DATA_PATH = "/data/sandboxes/"+SANDBOX_NAME+"/data/data/"



# Spark ML Problema de Regresión

En este notebook abordaremos el problema de Machine Learning Supervisado de regresión. Trabajaremos con el dataset Boston Housing que contiene información sobre las diferentes características de casas en la ciudad de Boston. Utilizaremos como variable objetivo el precio de las casas. Accederemos a este a través de la librería de ML de Python scikit-learn. Ajustaremos distintos modelos y compararemos los resultados obtenidos en éstos.



### Crear SparkSession
Nota: en Datio no es necesario crear la sesión de Spark ya al iniciar un notebook con el kernel PySpark Python3 - Spark 2.1.0 se crea automáticamente.

In [ ]:
# Respuesta
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()



### Cargamos los datos en un DataFrame de Spark
Cargamos los datos de scikit-learn y los consolidamos en un DataFrame de Spark.

In [ ]:
# Respuesta

import pandas as pd
from sklearn.datasets import load_boston

boston_dataset = load_boston()
boston = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
# We add the target variable to our Pandas DataFrame
boston['MEDV'] = boston_dataset.target
# We create a Spark DataFrame from the Pandas DataFrame
boston = spark.createDataFrame(boston)
boston.show()



Vemos el schema.

In [ ]:
# Respuesta
boston.printSchema()



Podemos ver la descripción de las variables del objeto cargado de scikit-learn.

In [ ]:
# Respuesta

print(boston_dataset.DESCR)



### Pasos previos

#### Vector Assembler
Para ajustar un modelo en Spark necesitamos indicar qué variables se van a utilizar como variables independientes. A través del parámetro _featuresCol_ de los distintos algoritmos, se le indica la variable que contiene la salida del  VectorAssembler con las variables independientes. 

Hacemos el VectorAssembler con todas las variables con excepcion del target.

In [ ]:
# Respuesta
from pyspark.ml.feature import VectorAssembler

variables_vector_assembler = list(set(boston.columns) - set(['MEDV'])) # We do not add the target variable
vectorassembler = VectorAssembler(inputCols = variables_vector_assembler, outputCol = 'assembled_features')
boston = vectorassembler.transform(boston)
boston.show()



#### División train/test
Realizamos la division train/test (o train/validation) para medir el desempeño del modelo tras el ajuste.

In [ ]:
# Respuesta

boston_train, boston_test = boston.randomSplit([0.8,0.2])



### Regresión Lineal
Modelo matemático usado para aproximar la relación de dependencia entre una variable dependiente $Y$, las variables independientes $X_i$ y un término aleatorio $ε$. Se expresa a traves de la siguiente ecuación:

$$Y = \beta_0+\beta_1*X_1+\beta_2*X_2+...+\beta_p*X_p+ε$$

donde 
- $Y$ es la variable dependiente, explicada o target,
- $X_i$ son las variables explicativas, independientes o regresoras,
- $\beta_i$ son los parametros que miden la influencia que tienen las variables explicativas sobre el target,

para todo $0<=i<=p$.

Ajustamos un modelo de regresión y sacamos los valores reales y las predicciones.

In [ ]:
# Respuesta
from pyspark.ml.regression import LinearRegression

linear_regression = LinearRegression(featuresCol='assembled_features', labelCol='MEDV')
linear_regression_model = linear_regression.fit(boston_train)
boston_test_linear_regression = linear_regression_model.transform(boston_test)
boston_test_linear_regression.select(['MEDV','prediction']).show()



### Arbol de Decisión
Modelo predictivo que se construye ejecutando una partición binaria recursiva de los datos identificando las variables y los puntos de corte de las mismas que mejor determinan el valor de la variable objetivo. En el entrenamiento son importantes los parámetros: medida de impureza y criterio de parada (normalmente profundidad).

Ajustamos un árbol de decisión para nuestro conjunto de datos y sacamos los valores reales y los predichos.

In [ ]:
# Respuesta

from pyspark.ml.regression import DecisionTreeRegressor

decision_tree_regression = DecisionTreeRegressor(featuresCol='assembled_features', labelCol='MEDV')
decision_tree_regression_model = decision_tree_regression.fit(boston_train)
boston_test_decision_tree_regression = decision_tree_regression_model.transform(boston_test)
boston_test_decision_tree_regression.select(['MEDV','prediction']).show()



### Random Forest
Modelo predictivo basado en árboles de decisión. Construye varios árboles tomando muestras del conjunto de datos (boosting) y muestras del conjunto de variables. Realiza la predicción para cada nuevo registro pasandolo por cada uno de los árboles y promediando los resultados obtenidos.

Ajustamos un random forest y obtenemos los valores reales y los predichos. 

In [ ]:
# Respuesta
from pyspark.ml.regression import RandomForestRegressor

random_forest = RandomForestRegressor(featuresCol='assembled_features', labelCol='MEDV')
random_forest_model = random_forest.fit(boston_train)
boston_test_random_forest = random_forest_model.transform(boston_test)
boston_test_random_forest.select(['MEDV','prediction']).show()

 

# Evaluación de modelos



Para comparar el desempeño de los modelos ajustados y poder seleccionar el mejor de ellos disponemos de diversas métricas. Algunas de ellas son:
- Error cuadrático medio (MSE)
- Raíz del error cuadrático medio (RMSE)
- R cuadrado (R²)
- Error absoluto medio (MAE)

Como ejemplo obtenemos el RMSE y el MAE de los modelos ajustados.

In [ ]:
# Respuesta

from pyspark.ml.evaluation import RegressionEvaluator

""" Possibilities::
metric name in evaluation - one of:
                       rmse - root mean squared error (default)
                       mse - mean squared error
                       r2 - r^2 metric
                       mae - mean absolute error.
"""

rmse = RegressionEvaluator(predictionCol='prediction', labelCol='MEDV', metricName='rmse')
mae = RegressionEvaluator(predictionCol='prediction', labelCol='MEDV', metricName='mae')



Imprimir métricas para los distintos modelos

In [ ]:
# Respuesta

print("RMSE for a linear regression: {}".format(rmse.evaluate(boston_test_linear_regression)))
print("RMSE for a decision tree: {}".format(rmse.evaluate(boston_test_decision_tree_regression)))
print("RMSE for a random forest: {}\n".format(rmse.evaluate(boston_test_random_forest)))

print("MAE for a linear regression: {}".format(mae.evaluate(boston_test_linear_regression)))
print("MAE for a decision tree: {}".format(mae.evaluate(boston_test_decision_tree_regression)))
print("MAE for a random forest: {}".format(mae.evaluate(boston_test_random_forest)))



Observando los valores obtenidos en las métricas de evaluación de modelos decidiríamos quedarnos con el random forest.



Podemos crear una función que tenga como parámetros de entrada el dataframe transformado, la columna de predicción, y el target, y devuelva un diccionario con todas las métricas disponibles y sus respectivos valores.

In [ ]:
def calculate_metrics(dataset, predictionCol='prediction', labelCol='MEDV'):
    
    metrics = RegressionEvaluator(predictionCol=predictionCol, labelCol=labelCol)
    
    rmse = metrics.evaluate(dataset, {metrics.metricName: "rmse"})
    mae = metrics.evaluate(dataset, {metrics.metricName: "mae"})
    mse = metrics.evaluate(dataset, {metrics.metricName: "mse"})
    r2 = metrics.evaluate(dataset, {metrics.metricName: "r2"})
    
    return {'rmse': rmse, 'mae':mae, 'mse': mse, 'r2':r2}

In [ ]:
calculate_metrics(boston_test_linear_regression)

In [ ]:
calculate_metrics(boston_test_decision_tree_regression)

In [ ]:
calculate_metrics(boston_test_random_forest)